# **SIC Capstone Project**

*Supermarket Project*

### **Abstract Data Structures**

In [1]:
class Queue:
    
    def __init__(self):
        self.queue = []

    def __str__(self):
        return "{}".format(self.queue)

    def is_empty(self):
        return len(self.queue) == 0

    def size(self):
        return len(self.queue)

    def enqueue(self, item):
        self.queue.append(item)

    def dequeue(self):
        return None if self.is_empty() else self.queue.pop(0)

    def peek_first(self):
        return None if self.is_empty() else self.queue[0]

    def peek_last(self):
        return None if self.is_empty() else self.queue[-1]

### **Custom Exceptions**

In [2]:
class ClientNotFoundError(Exception):
    '''Triggers when the client_name is not in the agents dictionary'''
    def __init__(self, client_name):
        self.message = f"Client {client_name} not found"
        super().__init__(self.message)

class SupermarketNotFoundError(Exception):
    '''Triggers when the supermarket_name is not in the agents dictionary'''
    def __init__(self, supermarket_name):
        self.message = f"Supermarket {supermarket_name} not found"
        super().__init__(self.message)

class ClientNotInSupermarketError(Exception):
    '''Triggers when the client is not in a supermarket'''
    def __init__(self, client_name):
        self.message = f"Client {client_name} is not in a supermarket at this moment"
        super().__init__(self.message)

class ClientAlreadyInASupermarketError(Exception):
    '''Triggers when the client is already in a supermarket'''
    def __init__(self, client_name, supermarket_name):
        self.message = f"Client {client_name} is already in a supermarket ({supermarket_name}) at this moment.",
        super().__init__(self.message)

class SupermarketClosedError(Exception):
    '''Triggers when the supermarket is closed when trying to execute some commands'''
    def __init__(self, supermarket_name):
        self.message = f"Supermarket {supermarket_name} is closed at this moment."
        super().__init__(self.message)

class ClientNotInLineError(Exception):
    '''Triggers when the client is not at the checkout line'''
    def __init__(self, client_name):
        self.message = f"Client {client_name} is not at the checkout line at this moment."
        super().__init__(self.message)

class ClientInCheckoutLineError(Exception):
    '''Triggers when the client is at the checkout line and is trying to execute certain commands'''
    def __init__(self, client_name):
        self.message = f"Can't perform this action. Client {client_name} is at the checkout line at this moment"
        super().__init__(self.message)

class ProductNotInInventoryError(Exception):
    '''Triggers when the specified product is not in the supermarket inventory'''
    def __init__(self, product_name, supermarket_name):
        self.message = f"Can't perform this action. {product_name} is not in {supermarket_name}'s inventory"
        super().__init__(self.message)

class ProductNotEnoughStockError(Exception):
    '''Triggers when the specified product doesn't have enough stock for the client to buy'''
    def __init__(self, supermarket_name, product_name, product_stock):
        self.message = f"Can't perform this action. Not enough stock of {product_name} in {supermarket_name}'s inventory. Current stock: {product_stock}"
        super().__init__(self.message)

class ProductNotInCartError(Exception):
    '''Triggers when the specified product is not in the client's cart'''
    def __init__(self, client_name, product_name):
        self.message = f"'{product_name} NOT in {client_name}'s cart"
        super().__init__(self.message)

class InvalidRemovalAmountError(Exception):
    '''Triggers when the amount of product to remove exceeds the current amount in the client's cart'''
    def __init__(self, client_name, product_name, current_amount):
        self.message = f"Can't remove product. {client_name}'s cart only has {current_amount} of '{product_name}'"
        super().__init__(self.message)

### **Class Hierarchy**

#### Agent

In [3]:
class Agent:
    """Clase base para todos los agentes en el sistema."""
    
    def __init__(self, name):
        self.name = name

    def describe(self):
        return f"Agent: {self.name}"

#### Client

In [14]:
from random import uniform
import json, copy

class Client(Agent):
    """Clase que representa a un cliente que interactúa con el supermercado."""

    def __init__(self, name):
        super().__init__(name)
        self.status  = { 
            "is_in_supermarket": False, 
            "supermarket": "", 
            "client_has_paid": False, 
            "wallet": round(uniform(1000.0, 5000.0), 2),
            "is_checking_out": False,
            "is_in_checkout_line": False
        }
        self.shopping_cart = { 
                
        }

    def move_to_supermarket(self, supermarket_name):
        self.status["is_in_supermarket"] = True
        self.status["supermarket"] = supermarket_name

    def pick_up_product(self, product_name, amount): # Al comprar un producto, debemos actualizar el inventario??
        if product_name in self.shopping_cart.keys():
            self.shopping_cart[product_name] += amount
        else:
            self.shopping_cart.update({ product_name: amount })

    def show_cart(self):
        cart = copy.deepcopy(self.shopping_cart)
        return f"{self.name}'s cart: { json.dumps(cart, sort_keys=True, indent=4) }"
    
    def remove_from_cart(self, product_name, amount):
        if self.shopping_cart[product_name] > amount:
            self.shopping_cart[product_name] -= amount
        elif self.shopping_cart[product_name] == amount:
            del self.shopping_cart[product_name]
    
    def is_in_supermarket(self):
        return self.status["is_in_supermarket"]
    
    def is_in_checkout_line(self):
        return self.status["is_in_checkout_line"]
    
    def is_product_in_cart(self, product_name):
        return product_name in self.shopping_cart.keys()

    def info(self):
        return f'''
            Client name: { self.name },
            Status: { json.dumps(self.status) },
            shopping_cart: { json.dumps(self.shopping_cart) },
        '''        

#### Supermarket

In [12]:
from random import uniform

class Supermarket(Agent):
    """Clase que representa un supermercado"""

    def __init__(self, name):
        super().__init__(name)
        self.status = {
            "clients": [], 
            "checkout_line": Queue(),
            "is_open": True
        }
        self.inventory = { "patatas": { "price": 12.90, "stock": 10} }
        
    def info(self):
        return f'''
            Supermarket name: {self.name}
            Status: {json.dumps(self.status)}
            Inventory: { json.dumps(self.inventory) }
        '''

    def add_product(self, product_name):
        """ Añade un nuevo producto al supermercado """
        product_name = product_name.capitalize() # Guardamos el nombre del producto capitalizado
        if product_name in self.inventory: #comprobamos si el producto se encuentra en el supermercado
            print(f'{product_name} already exist in {self.name}.')
        else:
            product_price = round(uniform(1.0, 50.0), 2)  #generamos un precio aleatorio entre 1.00 y 50.00
            product_stock = 0 # stock inicial por defecto
            self.inventory[product_name] = {
                "price" : product_price,
                "stock" : product_stock,
            }
            print(f'Product {product_name} added to {self.name} with price: {product_price} and stock: {product_stock}.')  
    
    def show_inventory(self):
        """ Muestra todo el inventario del supermercado """
        if not self.inventory:
            print(f'There are no products in {self.name}.')
        else: 
            print('----- Inventory -----')
            for product, details in self.inventory.items():
                price = details['price']
                stock = details['stock']
                print(f'Product: {product} | Price: {price}€ | Stock: {stock} units.')
    
    def show_products(self):
        """ Muestra la lista de productos del supermercado """
        if not self.inventory:
            print(f'There are no products in {self.name}.')
        else: 
            print('----- List of Products -----')
            for product in self.inventory.keys():
                print(f'- {product}')

    def restock_product(self, product_name, quantity): # Añadir comprobaciones de entrada del usuario
        """ Reabastecer un producto en el supermercado """
        if not product_name in self.inventory:
            print(f'Product: {product_name} not found in {self.name}')
        else: 
            self.inventory[product_name]['stock'] += quantity # Sumamos a la cantidad actual el restock
            new_stock = self.inventory[product_name]['stock']
            print(f'Product: {product_name} restocked. New Stock: {new_stock} units.')
    
    def set_open_close(self):
        """ Abrir/Cerrar el supermercado si no hay clientes dentro """
        if not self.status['clients']:
            if not self.status['is_open']:
                print(f'Supermarket {self.name} is already closed.')
            else:
                self.status['is_open'] = False
                print(f'Supermarket {self.name} is now closed.')

    def show_clients(self):
        """ Muestra la lista de clientes en el supermercado """
        if not self.status['clients']:
            print(f'There are no clients in {self.name}.')
        else:
            print('----- List of Clients -----')
            for client in self.status['clients']:
                print(f'- {client}') # Comprobar si el nombre viene capitalizado
    
    def show_clients_in_cart(self):
        """ Mostrar la lista de clientes en la caja """
        if not self.status['checkout_line'].queue:
            print('There are no clients in line yet.')
        else:
            print('----- Clients in Cart -----')
            for client in self.status['checkout_line'].queue:
                print(f'- {client}')

    def serve_next_customer(self):
        """ Atender al siguiente cliente en la caja """
        if not self.status['checkout_line'].is_empty: # Comprobamos si quedan clientes en la cola
            print('The are no clients in line.')
        else:
            client = self.status['checkout_line'].dequeue() # Extraemos al siguiente de la cola
            print(f'Attending to client: {client}')


#### CitySimulation

In [9]:
class CitySimulation:
    """Clase principal para gestionar la simulación de la ciudad."""
    
    def add_agent(self, agent_type, agent_name):
        """Añade un nuevo agente al sistema."""
        if agent_type == 'client':
            agents[agent_name] = Client(agent_name)
        elif agent_type == 'supermarket':
            agents[agent_name] = Supermarket(agent_name)
        print(f'{agent_type.capitalize()} {agent_name} added to the system.')

    def remove_agent(self, agent_name):
        """Elimina un agente del sistema."""
        if agent_name in agents:
            del agents[agent_name]
            print(f'Agent {agent_name} removed from the system.')
        else:
            print(f'Agent {agent_name} not found.')

    def list_agents(self):
        """Muestra todos los agentes en el sistema."""
        print("Current agents:")
        for agent in agents.values():
            print(agent.info())

    def move(self, client_name, supermarket_name):
        """Mover a un cliente a un supermercado específico"""
        client = agents[client_name]
        supermarket = agents[supermarket_name]
        client.move_to_supermarket(supermarket_name)
        supermarket.status["clients"].append(client)

    def pick_up(self, client_name, product_name, amount):
        """Permitir que un cliente recoja un producto y lo añada a la cesta"""
        client = agents[client_name]
        client.pick_up_product(product_name, int(amount))
        supermarket = agents[client.status["supermarket"]]
        supermarket.inventory[product_name]["stock"] -= int(amount)

    def buy(self, client_name, product_name):
        """Comprar un producto al pasar por caja"""
        pass

    def show_cart(self, client_name):
        """Mostrar los productos en la cesta del cliente"""
        client = agents[client_name]
        print(client.show_cart())

    def check_cart(self, client_name, product_name):
        client = agents[client_name]
        if client.is_product_in_cart(product_name):
            print(f"'{product_name}' is in {client_name}'s cart. Current amount: {client.shopping_cart[product_name]}")
        else:
            print(f"'{product_name}' is NOT in {client_name}'s cart")

    def remove_from_cart(self, client_name, product_name, amount):
        client = agents[client_name]
        client.remove_from_cart(product_name, amount)
        print(f"{amount} of '{product_name}' removed from {client_name}'s cart.")

    def client_exists(self, client_name):
        """Comprueba si el usuario existe en el diccionario de agentes"""
        return client_name in agents and isinstance(agents[client_name], Client)
    
    def supermarket_exists(self, supermarket_name):
        """Comprueba si el supermercado existe en el diccionario de agentes"""
        return supermarket_name in agents and isinstance(agents[supermarket_name], Supermarket)
    
    def is_client_in_supermarket(self, client_name):
        """Comprueba si el cliente se encuentra en un supermercado"""
        return agents[client_name].is_in_supermarket()
    
    def is_client_in_checkout_line(self, client_name):
        return agents[client_name].is_in_checkout_line()
    
    def is_supermarket_closed(self, supermarket_name):
        """Comprueba si el supermercado está cerrado"""
        return agents[supermarket_name].status["is_open"] == False
    
    def is_product_in_cart(self, client_name, product_name):
        """Comprueba si el producto especificado se encuentra en el carrito del cliente"""
        return product_name in agents[client_name].shopping_cart.keys()
    
    def amount_exceeded(self, client_name, product_name, amount):
        """Comprueba si la cantidad de producto que se quiere eliminar del carrito excede la cantidad que hay en el carrito"""
        return agents[client_name].shopping_cart[product_name] < amount
    
    def product_in_stock(self, product_name, supermarket_name):
        """Comprueba si el producto se encuentra en el inventario del supermercado"""
        return product_name in agents[supermarket_name].inventory.keys()
    
    def not_enough_stock(self, supermarket_name, product_name, amount):
        """Comprueba si no hay suficiente stock de un producto"""
        return agents[supermarket_name].inventory[product_name]["stock"] < int(amount)

    def help(self):
        """Muestra la lista de comandos disponibles."""
        print("""
            Available commands:
            - supermarket add <supermarket_name>: Agregar un nuevo supermercado al sistema.
            - supermarket add_product <supermarket_name> <product_name>: Agregar un producto al inventario del supermercado.
            - client add <client_name>: Agregar un cliente al sistema.
            - client move <client_name> <to_supermarket>: Mover a un cliente a un supermercado específico.
            - client pick_up <client_name> <product_name>: Permitir que un cliente recoja un producto.
            - supermarket show_clients <supermarket_name>: Mostrar la lista de clientes en el supermercado.
            - supermarket show_products <supermarket_name>: Mostrar la lista de productos en el supermercado.
            - supermarket restock <supermarket_name> <product_name> : Reabastecer un producto en el supermercado.
            - client buy <client_name> <product_name>: Comprar un producto y agregarlo a la cesta del cliente.
            - client show_cart <client_name>: Mostrar los productos en la cesta del cliente.
            - client checkout <client_name>: Preparar al cliente para pagar en caja.
            - supermarket show_clients_in_cash <supermarket_name>: Mostrar la lista de clientes en la caja.
            - supermarket serve_next_customer <supermarket_name>: Atender al siguiente cliente en la fila de caja.
            - client leave <client_name>: Permitir que un cliente salga del supermercado, siempre que haya pagado.
            - client leave_without_checkout <client_name>: Permitir que un cliente salga sin comprar, si su cesta está vacía.
            - client check_cart <client_name> <product_name>: Verificar si un producto está en la cesta del cliente.
            - supermarket set_open_close <supermarket_name> <open/close>: Abrir / Cerrar el supermercado si no hay clientes dentro.
            - supermarket show_inventory <supermarket_name>: Mostrar todo el inventario del supermercado.
            - client remove_from_cart <client_name> <product_name>: Permitir que el cliente retire un producto de su cesta.
            - client get_in_line <client_name>: Permitir que un cliente se ponga en la fila para pagar.
            """)            

    def command_loop(self):
        """Bucle principal para gestionar comandos del usuario."""
        print("Starting city simulation... Type 'q' to exit")
        while True:
            command = input('> ')
            if command == 'q':
                break
            self.process_command(command)

    def process_command(self, command):
        """Procesa los comandos ingresados por el usuario."""
        parts = command.split()
        if not parts:
            return
        
        cmd = parts[0]
        
        if cmd == '?':
            self.help()  # Llama al método de ayuda
            return
        
        # supermarket commands
        elif cmd == 'supermarket':

            #********************************************************
            #***** Completar con las funcionalidades requeridas *****
            #********************************************************


            ### falta hacer comprobaciones antes de añadirlo al diccionario
            if parts[1] == 'add':
                try:
                    _, _, supermarket_name = parts
                    self.add_agent("supermarket", supermarket_name)
                except ValueError:
                    print("Error: Invalid add command format. Use 'supermarket add <supermarket_name>'")

            elif parts[1] == 'add_product':
                pass

            elif parts[1] == 'show_clients':
                pass

            elif parts[1] == 'show_products':
                pass

            elif parts[1] == 'restock':
                pass

            elif parts[1] == 'show_clients_in_cart':
                pass

            elif parts[1] == 'serve_next_customer':
                pass

            elif parts[1] == 'set_open_close':
                pass

            elif parts[1] == 'show_inventory':
                pass



        # client commands
        elif cmd == 'client':

            ### falta hacer comprobaciones antes de añadirlo al diccionario
            if parts[1] == 'add':
                try:
                    _, _, client_name = parts
                    self.add_agent('client', client_name)
                except ValueError:
                    print("Error: Invalid add command format. Use 'client add <client_name>'")

            elif parts[1] == 'remove_client':
                try:
                    _, _, client_name = parts
                    self.remove_agent(client_name)
                except ValueError:
                    print("Error: Invalid remove_client command format. Use 'client remove_client <client_name>'")

            #********************************************************
            #***** Completar con las funcionalidades requeridas *****
            #********************************************************
            
            elif parts[1] == 'move':
                try:
                    _, _, client_name, supermarket_name = parts
                    if not self.client_exists(client_name):
                        raise ClientNotFoundError(client_name)                                                 
                    if not self.supermarket_exists(supermarket_name):
                        raise SupermarketNotFoundError(supermarket_name)                                                       
                    if self.is_supermarket_closed(supermarket_name):
                        raise SupermarketClosedError(supermarket_name)                         
                    if self.is_client_in_supermarket(client_name):
                        raise ClientAlreadyInASupermarketError(client_name, agents[client_name].status["supermarket"])                    
                except ValueError:
                    print("Error: Invalid move command format. Use 'client move <client_name> <supermarket_name>'")
                except ClientNotFoundError as e:
                    print(e)
                except SupermarketNotFoundError as e:
                    print(e)
                except SupermarketClosedError as e:
                    print(e)
                except ClientAlreadyInASupermarketError as e:
                    print(e)
                else:
                    self.move(client_name, supermarket_name)
                    print(f"Client {client_name} moved to {supermarket_name}")

            elif parts[1] == 'pick_up':
                try:
                    _, _, client_name, product_name, amount = parts
                    if not self.client_exists(client_name):
                        raise ClientNotFoundError(client_name)
                    if not self.is_client_in_supermarket(client_name):
                        raise ClientNotInSupermarketError(client_name)
                    if self.is_client_in_checkout_line(client_name):
                        raise ClientInCheckoutLineError(client_name)
                    if not self.product_in_stock(product_name, agents[client_name].status["supermarket"]):
                        raise ProductNotInInventoryError(product_name, agents[client_name].status["supermarket"])
                    if self.not_enough_stock(agents[client_name].status["supermarket"], product_name, amount):
                        supermarket = agents[agents[client_name].status["supermarket"]]
                        product_stock = supermarket.inventory[product_name]["stock"]
                        raise ProductNotEnoughStockError(agents[client_name].status["supermarket"], product_name, product_stock)
                except ValueError:
                    print("Error: Invalid pick_up command format. Use 'client pick_up <client_name> <product_name> <amount>'")
                except ClientNotFoundError as e:
                    print(e) 
                except ClientNotInSupermarketError as e:
                    print(e)              
                except ClientInCheckoutLineError as e:
                    print(e)     
                except ProductNotInInventoryError as e:
                    print(e)    
                except ProductNotEnoughStockError as e:
                    print(e)               
                else:
                    self.pick_up(client_name, product_name, amount)
                    print(f"{amount} of '{product_name}' added to {client_name}'s cart")

            elif parts[1] == 'pay':
                pass

            elif parts[1] == 'show_cart':
                try:
                    _, _, client_name = parts
                    if not self.client_exists(client_name):
                        raise  ClientNotFoundError(client_name)                   
                    if not self.is_client_in_supermarket(client_name):
                        raise ClientNotInSupermarketError(client_name)                   
                except ValueError:
                    print("Error: Invalid show_cart command format. Use 'client show_cart <client_name>'")
                except ClientNotFoundError as e:
                    print(e)
                except ClientNotInSupermarketError as e:
                    print(e)                        
                else:
                    self.show_cart(client_name)

            elif parts[1] == 'exit_line':
                pass

            elif parts[1] == 'leave':
                pass

            elif parts[1] == 'leave_without_checkout':
                pass

            elif parts[1] == 'check_cart':
                try:
                    _, _, client_name, product_name = parts
                except ValueError:
                    print("Error: Invalid check_cart command format. Use 'client check_cart <client_name> <product_name>'")
                except ClientNotFoundError as e:
                    print(e)
                except ClientNotInSupermarketError as e:
                    print(e)
                else:
                    self.check_cart(client_name, product_name)

            elif parts[1] == 'remove_from_cart':
                try:
                    _, _, client_name, product_name, amount = parts
                    amount = int(amount)
                    if not self.client_exists(client_name):
                        raise ClientNotFoundError(client_name)
                    if not self.is_client_in_supermarket(client_name):
                        raise ClientNotInSupermarketError(client_name)
                    if not self.is_product_in_cart(client_name, product_name):
                        raise ProductNotInCartError(client_name, product_name)
                    if self.amount_exceeded(client_name, product_name, amount):
                        raise InvalidRemovalAmountError(client_name, product_name, agents[client_name].shopping_cart[product_name])
                except ValueError:
                    print("Error: Invalid remove_from_cart command format. Use 'client remove_from_cart <client_name> <product_name> <amount>'")
                except ClientNotFoundError as e:
                    print(e)
                except ClientNotInSupermarketError as e:
                    print(e)
                except ProductNotInCartError as e:
                    print(e)
                except InvalidRemovalAmountError as e:
                    print(e)                   
                else:
                    self.remove_from_cart(client_name, product_name, amount)

            elif parts[1] == 'get_in_line':
                pass

        else:
            print("Unknown command. Type 'help' for a list of commands.")

### **Diccionario global para almacenar agentes**

In [15]:
agents = {}

### **Main program**

In [16]:
if __name__ == "__main__":
    simulation = CitySimulation()
    simulation.command_loop()

Starting city simulation... Type 'q' to exit
Client Andrei added to the system.
Supermarket Mercadona added to the system.
Client Andrei moved to Mercadona
Andrei's cart: {}
2 of patatas added to Andrei's cart
Andrei's cart: {
    "patatas": 2
}
1 of 'patatas' removed from Andrei's cart.
Andrei's cart: {
    "patatas": 1
}
'cebollas' is NOT in Andrei's cart
'patatas' is in Andrei's cart. Current amount: 1
